# Health Insurance Risk Classifier
This project classifies individuals into health insurance risk categories based on their health metrics.
It uses a simple rule-based approach to categorize risk levels.

Author Carmine Giardino

## Step 1: Import Necessary Libraries

Import all required Python libraries for data manipulation, analysis, database operations, and visualization.


In [ ]:
# Import pandas for data handling
# Ref: https://pandas.pydata.org/docs/user_guide/index.html
import pandas as pd
# Import numpy for numerical operations
# Ref: https://numpy.org/doc/stable/user/absolute_beginners.html
import numpy as np
# Import sqlite3 for database operations
# Ref: https://docs.python.org/3/library/sqlite3.html
import sqlite3
# Import matplotlib for plotting
# Ref: https://matplotlib.org/stable/tutorials/introductory/pyplot.html
import matplotlib.pyplot as plt
# Import seaborn for advanced visualizations
# Ref: https://seaborn.pydata.org/tutorial.html
import seaborn as sns

## Step 2: Read the Health Insurance Data from CSV File

Load the health insurance dataset from a CSV file into a pandas DataFrame for analysis.


In [ ]:
# Read CSV file into a pandas DataFrame
# Ref: https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html
path = './data/health_insurance_data.csv'
df = pd.read_csv(path)
print("Data loaded successfully.")
# Display first 5 rows to understand the data structure
# Ref: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html
print(df.head())

## Step 3: Clean the Data by Handling Missing Values

Handle missing values in the dataset using appropriate imputation strategies: mean for numeric columns and mode for categorical columns.


In [ ]:
# Fill missing values in numeric columns with mean
# This is a common imputation strategy for numerical data
# Ref: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html
for col in df.select_dtypes(include=[np.number]).columns:
    df[col] = df[col].fillna(df[col].mean())

# Fill missing values in non-numeric columns with mode
# Mode is the most frequent value, suitable for categorical data
# Ref: https://pandas.pydata.org/docs/reference/api/pandas.Series.mode.html
for col in df.select_dtypes(exclude=[np.number]).columns:
    mode = df[col].mode()[0] if not df[col].mode().empty else None
    df[col] = df[col].fillna(mode)

print("Missing values handled.")
# Check for remaining missing values
# Ref: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.isnull.html
print(df.isnull().sum())

### Step 3.1: Encode Categorical Variables

Convert categorical text variables (sex, smoker, region) into numerical format using binary encoding and one-hot encoding for use in analysis.


In [ ]:
# Encode categorical variables for analysis
# Machine learning models require numerical input, so we convert text categories to numbers
# sex: male=1, female=0 (binary encoding)
# Ref: https://pandas.pydata.org/docs/reference/api/pandas.Series.map.html
df['sex_encoded'] = df['sex'].map({'male': 1, 'female': 0})

# smoker: yes=1, no=0 (binary encoding)
df['smoker_encoded'] = df['smoker'].map({'yes': 1, 'no': 0})

# region: use one-hot encoding
# One-hot encoding converts categorical text into a numerical format without implying any ordinal relationship between regions.
# Each region becomes a separate binary column
# Ref: https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html
region_dummies = pd.get_dummies(df['region'], prefix='region')
# Concatenate the one-hot encoded columns with the original DataFrame
# Ref: https://pandas.pydata.org/docs/reference/api/pandas.concat.html
df = pd.concat([df, region_dummies], axis=1)

print("Categorical variables encoded.")
print(df.head())
print(f"\nDataset shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")

## Step 4: Normalize Numeric Features for Better Comparison

Apply min-max normalization to scale numeric features to a [0, 1] range, making them comparable and suitable for analysis.


In [ ]:
# Preserve original charges for later correlation analysis
# Ref: https://pandas.pydata.org/docs/reference/api/pandas.Series.copy.html
df['charges_original'] = df['charges'].copy()

# Identify numeric columns
# Ref: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.select_dtypes.html
numeric_cols = df.select_dtypes(include=[np.number]).columns
# Exclude charges_original from normalization
numeric_cols = [col for col in numeric_cols if col != 'charges_original']

# Normalize using manual min-max scaling (scales values to range [0, 1])
# Formula: normalized = (value - min) / (max - min)
# Ref: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html
# Ref: https://en.wikipedia.org/wiki/Feature_scaling#Rescaling_(min-max_normalization)
for col in numeric_cols:
    min_val = df[col].min()
    max_val = df[col].max()
    df[col] = (df[col] - min_val) / (max_val - min_val)
print("Numeric features normalized.")
print(df.head())

### Step 4.1: Save the Data to SQLite Database

Save both the original and processed datasets to a SQLite database to ensure data persistence and enable SQL-based queries.


In [ ]:
# Create SQLite database connection
db_path = './data/health_insurance.db'
# Connect to SQLite database (it will be created if it doesn't exist)
# Ref: https://docs.python.org/3/library/sqlite3.html#sqlite3.connect
conn = sqlite3.connect(db_path)

# Save the processed data to SQLite
# Ref: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html
df.to_sql('health_insurance_processed', conn, if_exists='replace', index=False)

# Also save the original data (before normalization) if needed
df_original = pd.read_csv(path)
df_original.to_sql('health_insurance_original', conn, if_exists='replace', index=False)

# Close the connection
# Ref: https://docs.python.org/3/library/sqlite3.html#sqlite3.Connection.close
conn.close()

print(f"Data saved to SQLite database at: {db_path}")
print("Tables created: 'health_insurance_original' and 'health_insurance_processed'")

## Step 5: Define Risk Classification Rules Based on Health Metrics

Create a rule-based classification function that assigns risk scores based on age, BMI, smoking status, and number of children, then categorizes individuals into Low, Medium, or High risk groups.


In [ ]:
# The function assigns weights to different factors to compute a risk score and classifies into 'Low', 'Medium', 'High' risk categories.
# This is a rule-based classification system (expert system approach)
# Ref: https://en.wikipedia.org/wiki/Rule-based_system
# Ref: https://www.sciencedirect.com/topics/computer-science/rule-based-classification
def classify_risk(row):
    risk_score = 0
    
    # Age scoring - higher impact for older ages
    if row['age'] > 0.7:
        risk_score += 4
    elif row['age'] > 0.4:
        risk_score += 2
    elif row['age'] > 0.2:
        risk_score += 1
    
    # BMI scoring - higher impact for obesity
    if row['bmi'] > 0.7:
        risk_score += 4
    elif row['bmi'] > 0.5:
        risk_score += 2
    elif row['bmi'] > 0.3:
        risk_score += 1
    
    # Smoker - very high impact
    if row['smoker'] == 'yes':
        risk_score += 5
    
    # Children - lower impact (indicates family responsibility)
    if row['children'] >= 3:
        risk_score += 1
    elif row['children'] >= 1:
        risk_score += 0.5
    
    if risk_score >= 7:
        return 'High'
    elif risk_score >= 4:
        return 'Medium'
    else:
        return 'Low'


## Step 6: Apply Risk Classification to All Records

Apply the risk classification function to each individual record in the dataset and examine the distribution of risk categories.


In [ ]:
# Apply the risk classification function to each row
# axis=1 means apply function to each row (rather than each column)
# Ref: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html
# Ref: https://pandas.pydata.org/docs/user_guide/basics.html#row-or-column-wise-function-application
df['risk_category'] = df.apply(classify_risk, axis=1)

print("Risk classification applied.")
print(df[['age', 'bmi', 'smoker', 'children', 'risk_category']].head(10))
print(f"\nRisk category distribution:")
# Count occurrences of each category
# Ref: https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html
print(df['risk_category'].value_counts())
print(f"\nPercentage distribution:")
# normalize=True converts counts to proportions
print(df['risk_category'].value_counts(normalize=True) * 100)

## Step 7: Save the Final Dataset with Risk Classifications to SQLite Database

Store the processed dataset with risk classifications in a SQLite database for persistence and future analysis.


In [ ]:
# Reconnect to SQLite database
# Ref: https://docs.python.org/3/library/sqlite3.html#sqlite3.connect
conn = sqlite3.connect(db_path)

# Save the final dataset with risk classifications
# if_exists='replace' will drop the table if it exists and create a new one
# Ref: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html
# Ref: https://docs.python.org/3/library/sqlite3.html#sqlite3.Connection.execute
df.to_sql('health_insurance_with_risk', conn, if_exists='replace', index=False)

print(f"Dataset with risk classifications saved to 'health_insurance_with_risk' table")
print(f"Total records: {len(df)}")

## Step 8: Run SQL Queries to Confirm Correct Storage and Retrieval

Execute SQL queries to verify that data was correctly stored in the database and demonstrate data retrieval capabilities.


In [ ]:
# Query 1: Count total records
query1 = "SELECT COUNT(*) as total_records FROM health_insurance_with_risk"
# Run the query and fetch results into a DataFrame
# Ref: https://pandas.pydata.org/docs/reference/api/pandas.read_sql_query.html
result1 = pd.read_sql_query(query1, conn)
print("Query 1 - Total records:")
print(result1)
print()

# Query 2: Count by risk category
# Group by risk category and count occurrences
# Ref: https://www.sqltutorial.org/sql-group-by/
query2 = """
SELECT risk_category, COUNT(*) as count 
FROM health_insurance_with_risk 
GROUP BY risk_category 
ORDER BY count DESC
"""
result2 = pd.read_sql_query(query2, conn)
print("Query 2 - Risk category distribution:")
print(result2)
print()

# Query 3: Average age and BMI by risk category
# Ref: https://www.sqltutorial.org/sql-aggregate-functions/sql-avg/
query3 = """
SELECT risk_category, 
       AVG(age) as avg_age, 
       AVG(bmi) as avg_bmi,
       COUNT(*) as count
FROM health_insurance_with_risk 
GROUP BY risk_category
"""
result3 = pd.read_sql_query(query3, conn)
print("Query 3 - Average age and BMI by risk category:")
print(result3)
print()

# Query 4: Sample records from each risk category
query4 = """
SELECT risk_category, age, bmi, smoker, children, charges_original
FROM health_insurance_with_risk
ORDER BY risk_category, charges_original DESC
LIMIT 15
"""
result4 = pd.read_sql_query(query4, conn)
print("Query 4 - Sample records from database:")
print(result4)

# Close the connection
conn.close()
print("\nSQL queries completed successfully. Database connection closed.")


## Step 9: Load Data for Exploratory Data Analysis

Reload the processed dataset from the database and prepare visualization settings for comprehensive exploratory data analysis.


In [ ]:
# Set style for better-looking plots
# Ref: https://matplotlib.org/stable/tutorials/introductory/customizing.html
plt.style.use('default')
sns.set_palette("husl")

# Reconnect to SQLite database
conn = sqlite3.connect(db_path)

# Load the processed data with risk classifications
# Ref: https://pandas.pydata.org/docs/reference/api/pandas.read_sql_query.html
query = "SELECT * FROM health_insurance_with_risk"
df_analysis = pd.read_sql_query(query, conn)

print("Data loaded from SQLite for analysis.")
print(f"Dataset shape: {df_analysis.shape}")
print(f"\nFirst few rows:")
print(df_analysis.head())
print(f"\nData types:")
print(df_analysis.dtypes)

conn.close()


## Step 10: Visualize and Analyze the Data

Create comprehensive visualizations to explore the distribution of key features (age, BMI, insurance charges, risk categories, and smoker status) and their relationships with risk classifications.


### 10.1: Age Distribution Analysis

Examine the distribution of age across the dataset and compare age patterns between different risk categories using histograms and box plots.


In [ ]:
# Create figure with subplots for age distribution
# Ref: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.subplots.html
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram of age (normalized values)
# Ref: https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.hist.html
axes[0].hist(df_analysis['age'], bins=30, color='skyblue', edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Age (Normalized)', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_title('Distribution of Age (Normalized)', fontsize=14, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)
# Add annotation explaining the spike at 0.0
# Ref: https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.annotate.html
axes[0].annotate('Spike at 0.0 represents\nminimum age (18 years)', 
                 xy=(0.0, 0), xytext=(0.15, 0.7), 
                 textcoords='axes fraction',
                 bbox=dict(boxstyle='round,pad=0.5', facecolor='yellow', alpha=0.3),
                 fontsize=9)

# Box plot of age by risk category
# Ref: https://seaborn.pydata.org/generated/seaborn.boxplot.html
sns.boxplot(data=df_analysis, x='risk_category', y='age', ax=axes[1], 
            order=['Low', 'Medium', 'High'])
axes[1].set_xlabel('Risk Category', fontsize=12)
axes[1].set_ylabel('Age (Normalized)', fontsize=12)
axes[1].set_title('Age Distribution by Risk Category', fontsize=14, fontweight='bold')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("Age distribution analysis:")
print(f"Mean age (normalized): {df_analysis['age'].mean():.3f}")
print(f"Median age (normalized): {df_analysis['age'].median():.3f}")
print(f"Std deviation: {df_analysis['age'].std():.3f}")


### 10.2: BMI Distribution Analysis

Analyze the Body Mass Index (BMI) distribution and its relationship with risk categories to understand health status patterns.


In [ ]:
# Create figure with subplots for BMI distribution
# BMI (Body Mass Index) is a measure of body fat based on height and weight
# Ref: https://www.cdc.gov/healthyweight/assessing/bmi/index.html
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram of BMI (normalized values)
axes[0].hist(df_analysis['bmi'], bins=30, color='skyblue', edgecolor='black', alpha=0.7)
axes[0].set_xlabel('BMI (Normalized)', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_title('Distribution of BMI (Normalized)', fontsize=14, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)

# Box plot of BMI by risk category
# Box plots show median, quartiles, and outliers
# Ref: https://seaborn.pydata.org/generated/seaborn.boxplot.html
sns.boxplot(data=df_analysis, x='risk_category', y='bmi', ax=axes[1],
            order=['Low', 'Medium', 'High'])
axes[1].set_xlabel('Risk Category', fontsize=12)
axes[1].set_ylabel('BMI (Normalized)', fontsize=12)
axes[1].set_title('BMI Distribution by Risk Category', fontsize=14, fontweight='bold')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("BMI distribution analysis:")
print(f"Mean BMI (normalized): {df_analysis['bmi'].mean():.3f}")
print(f"Median BMI (normalized): {df_analysis['bmi'].median():.3f}")
print(f"Std deviation: {df_analysis['bmi'].std():.3f}")


### 10.3: Insurance Charges Distribution Analysis

Explore the distribution of insurance charges and identify how costs vary across different risk categories.


In [ ]:
# Create figure with subplots for charges distribution
# Insurance charges represent the cost of health insurance
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram of charges (original values)
# Using original values instead of normalized for better interpretability
axes[0].hist(df_analysis['charges_original'], bins=30, color='skyblue', edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Charges ($)', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_title('Distribution of Insurance Charges', fontsize=14, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)

# Box plot of charges by risk category
sns.boxplot(data=df_analysis, x='risk_category', y='charges_original', ax=axes[1],
            order=['Low', 'Medium', 'High'])
axes[1].set_xlabel('Risk Category', fontsize=12)
axes[1].set_ylabel('Charges ($)', fontsize=12)
axes[1].set_title('Insurance Charges by Risk Category', fontsize=14, fontweight='bold')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("Charges distribution analysis:")
print(f"Mean charges: ${df_analysis['charges_original'].mean():.2f}")
print(f"Median charges: ${df_analysis['charges_original'].median():.2f}")
print(f"Std deviation: ${df_analysis['charges_original'].std():.2f}")
print(f"Min charges: ${df_analysis['charges_original'].min():.2f}")
print(f"Max charges: ${df_analysis['charges_original'].max():.2f}")


### 10.4: Smoker Status Analysis

Investigate the impact of smoking status on risk categories and understand how smoking correlates with higher risk classifications.


In [ ]:
# Create figure for smoker analysis
# Smoking is a critical factor in health insurance risk assessment
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Smoker distribution
smoker_counts = df_analysis['smoker'].value_counts()
axes[0].bar(smoker_counts.index, smoker_counts.values, color=['lightblue', 'salmon'], 
            edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Smoker Status', fontsize=12)
axes[0].set_ylabel('Count', fontsize=12)
axes[0].set_title('Distribution of Smoker Status', fontsize=14, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)

# Add count labels
for i, (status, count) in enumerate(smoker_counts.items()):
    axes[0].text(i, count, str(count), ha='center', va='bottom', fontweight='bold')

# Smoker status by risk category
# Cross-tabulation shows relationship between two categorical variables
# Ref: https://pandas.pydata.org/docs/reference/api/pandas.crosstab.html
smoker_risk = pd.crosstab(df_analysis['smoker'], df_analysis['risk_category'])
smoker_risk[['Low', 'Medium', 'High']].plot(kind='bar', ax=axes[1], 
                                              color=['green', 'orange', 'red'],
                                              edgecolor='black', alpha=0.7)
axes[1].set_xlabel('Smoker Status', fontsize=12)
axes[1].set_ylabel('Count', fontsize=12)
axes[1].set_title('Risk Category by Smoker Status', fontsize=14, fontweight='bold')
axes[1].legend(title='Risk Category')
axes[1].grid(axis='y', alpha=0.3)
axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=0)

plt.tight_layout()
plt.show()

print("Smoker status analysis:")
print(smoker_risk)


### 10.5: Risk Category Distribution Analysis

Visualize the overall distribution of risk categories across the entire population using bar charts and pie charts.


In [ ]:
# Create figure for risk category distribution
# Visualize how individuals are distributed across risk categories
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Count plot of risk categories
# Bar chart showing frequency of each category
# Ref: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.bar.html
risk_counts = df_analysis['risk_category'].value_counts().sort_index()
colors = ['green', 'orange', 'red']
axes[0].bar(risk_counts.index, risk_counts.values, color=colors, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Risk Category', fontsize=12)
axes[0].set_ylabel('Count', fontsize=12)
axes[0].set_title('Distribution of Risk Categories', fontsize=14, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)

# Add count labels on bars
# Ref: https://matplotlib.org/stable/api/text_api.html#matplotlib.text.Text
for i, (category, count) in enumerate(risk_counts.items()):
    axes[0].text(i, count, str(count), ha='center', va='bottom', fontweight='bold')

# Pie chart of risk categories
# Shows proportional distribution
# Ref: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.pie.html
axes[1].pie(risk_counts.values, labels=risk_counts.index, autopct='%1.1f%%',
            colors=colors, startangle=90, explode=(0.05, 0.05, 0.05))
axes[1].set_title('Risk Category Proportions', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("Risk category distribution:")
print(risk_counts)
print("\nPercentages:")
print(df_analysis['risk_category'].value_counts(normalize=True) * 100)


## Step 11: Examine Correlations Between Variables

Analyze the relationships between different features using correlation matrices, scatter plots, and pairwise visualizations to identify key patterns and predictors.


### 11.1: Correlation Matrix

Calculate and visualize the correlation coefficients between all numeric features to identify strong relationships and potential predictors of insurance charges.


In [ ]:
# Select numeric columns for correlation analysis
# Correlation measures the linear relationship between two variables
# Exclude normalized versions if we want to see original relationships
numeric_features = ['age', 'bmi', 'children', 'sex_encoded', 'smoker_encoded', 'charges_original']
correlation_data = df_analysis[numeric_features].copy()

# Calculate correlation matrix
# Pearson correlation coefficient ranges from -1 (perfect negative) to +1 (perfect positive)
# Ref: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html
correlation_matrix = correlation_data.corr()

print("Correlation Matrix:")
print(correlation_matrix)
print("\n" + "="*80 + "\n")

# Heatmaps provide visual representation of correlation strength
# Ref: https://seaborn.pydata.org/generated/seaborn.heatmap.html
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8}, fmt='.3f')
plt.title('Correlation Matrix of Health Insurance Features', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

# Print strongest correlations with charges
print("Correlations with Insurance Charges (sorted by absolute value):")
charges_corr = correlation_matrix['charges_original'].drop('charges_original').sort_values(key=abs, ascending=False)
for feature, corr in charges_corr.items():
    print(f"  {feature:20s}: {corr:+.4f}")


### 11.2: Scatter Plots - Key Relationships

Create scatter plots to visualize bivariate relationships between key features and insurance charges, colored by smoker status and risk category.


In [ ]:
# Create scatter plots for key relationships
# Scatter plots help visualize relationships between two continuous variables
# Ref: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.scatter.html
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# Age vs Charges (colored by smoker status)
# Ref: https://matplotlib.org/stable/tutorials/colors/colormaps.html
scatter1 = axes[0, 0].scatter(df_analysis['age'], df_analysis['charges_original'],
                               c=df_analysis['smoker_encoded'], cmap='RdYlGn_r',
                               alpha=0.6, edgecolors='black', linewidth=0.5)
axes[0, 0].set_xlabel('Age (Normalized)', fontsize=11)
axes[0, 0].set_ylabel('Charges ($)', fontsize=11)
axes[0, 0].set_title('Age vs Charges (colored by Smoker Status)', fontsize=12, fontweight='bold')
axes[0, 0].grid(alpha=0.3)
cbar1 = plt.colorbar(scatter1, ax=axes[0, 0])
cbar1.set_label('Smoker (1=Yes, 0=No)', fontsize=10)

# BMI vs Charges (colored by smoker status)
scatter2 = axes[0, 1].scatter(df_analysis['bmi'], df_analysis['charges_original'],
                               c=df_analysis['smoker_encoded'], cmap='RdYlGn_r',
                               alpha=0.6, edgecolors='black', linewidth=0.5)
axes[0, 1].set_xlabel('BMI (Normalized)', fontsize=11)
axes[0, 1].set_ylabel('Charges ($)', fontsize=11)
axes[0, 1].set_title('BMI vs Charges (colored by Smoker Status)', fontsize=12, fontweight='bold')
axes[0, 1].grid(alpha=0.3)
cbar2 = plt.colorbar(scatter2, ax=axes[0, 1])
cbar2.set_label('Smoker (1=Yes, 0=No)', fontsize=10)

# Age vs BMI (colored by risk category)
risk_colors = {'Low': 0, 'Medium': 1, 'High': 2}
df_analysis['risk_numeric'] = df_analysis['risk_category'].map(risk_colors)
scatter3 = axes[1, 0].scatter(df_analysis['age'], df_analysis['bmi'],
                               c=df_analysis['risk_numeric'], cmap='RdYlGn_r',
                               alpha=0.6, edgecolors='black', linewidth=0.5)
axes[1, 0].set_xlabel('Age (Normalized)', fontsize=11)
axes[1, 0].set_ylabel('BMI (Normalized)', fontsize=11)
axes[1, 0].set_title('Age vs BMI (colored by Risk Category)', fontsize=12, fontweight='bold')
axes[1, 0].grid(alpha=0.3)
cbar3 = plt.colorbar(scatter3, ax=axes[1, 0], ticks=[0, 1, 2])
cbar3.set_ticklabels(['Low', 'Medium', 'High'])
cbar3.set_label('Risk Category', fontsize=10)

# Children vs Charges (colored by risk category)
scatter4 = axes[1, 1].scatter(df_analysis['children'], df_analysis['charges_original'],
                               c=df_analysis['risk_numeric'], cmap='RdYlGn_r',
                               alpha=0.6, edgecolors='black', linewidth=0.5, s=50)
axes[1, 1].set_xlabel('Children (Normalized)', fontsize=11)
axes[1, 1].set_ylabel('Charges ($)', fontsize=11)
axes[1, 1].set_title('Children vs Charges (colored by Risk Category)', fontsize=12, fontweight='bold')
axes[1, 1].grid(alpha=0.3)
cbar4 = plt.colorbar(scatter4, ax=axes[1, 1], ticks=[0, 1, 2])
cbar4.set_ticklabels(['Low', 'Medium', 'High'])
cbar4.set_label('Risk Category', fontsize=10)

plt.tight_layout()
plt.show()


### 11.3: Pairwise Relationships

Generate a comprehensive pairplot showing all pairwise relationships between key features, grouped by risk category to reveal multivariate patterns.


In [ ]:
# Create pairplot for key features
# Pairplots show pairwise relationships in a dataset
# Ref: https://seaborn.pydata.org/generated/seaborn.pairplot.html
print("Generating pairplot (this may take a moment)...")

# Select subset of features for pairplot
pairplot_features = ['age', 'bmi', 'children', 'charges_original', 'risk_category']
pairplot_data = df_analysis[pairplot_features].copy()

# Create pairplot
pairplot = sns.pairplot(pairplot_data, hue='risk_category', 
                        hue_order=['Low', 'Medium', 'High'],
                        palette={'Low': 'green', 'Medium': 'orange', 'High': 'red'},
                        diag_kind='hist', plot_kws={'alpha': 0.6, 'edgecolor': 'none'},
                        height=2.5, aspect=1.2)
pairplot.figure.suptitle('Pairwise Relationships Colored by Risk Category',
                      y=1.01, fontsize=16, fontweight='bold')

# Move legend outside and make it bigger with correct colors
# Ref: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.legend.html
from matplotlib.patches import Patch

# Remove the default legend that seaborn creates
# The legend is attached to one of the axes, so we need to find it and remove it
for ax in pairplot.figure.axes:
    legend = ax.get_legend()
    if legend is not None:
        legend.remove()
        break

# Create custom legend handles with correct colors
legend_handles = [
    Patch(color='green', label='Low'),
    Patch(color='orange', label='Medium'),
    Patch(color='red', label='High')
]

pairplot.figure.legend(handles=legend_handles,
                    title='Risk Category',
                    loc='center right',
                    bbox_to_anchor=(1, 0.5),
                    fontsize=14,
                    title_fontsize=16,
                    frameon=True,
                    fancybox=True,
                    shadow=True)

plt.tight_layout()
plt.subplots_adjust(right=0.85)  # Make room for the legend
plt.show()

print("Pairplot generated successfully.")


## Step 12: Summarize Key Findings

Compile and present the most important insights discovered through the exploratory data analysis, including risk distributions, correlations, and actionable observations.


In [ ]:
# Summarize the exploratory data analysis results
# This section provides a comprehensive overview of insights gained from the data
print("="*80)
print("KEY FINDINGS FROM EXPLORATORY DATA ANALYSIS")
print("="*80)
print()

# 1. Dataset Overview
print("1. DATASET OVERVIEW")
print("-" * 40)
print(f"   Total records: {len(df_analysis)}")
print(f"   Features analyzed: {len(numeric_features)}")
print(f"   Risk categories: {df_analysis['risk_category'].nunique()}")
print()

# 2. Risk Distribution
print("2. RISK CATEGORY DISTRIBUTION")
print("-" * 40)
risk_dist = df_analysis['risk_category'].value_counts().sort_index()
for category, count in risk_dist.items():
    percentage = (count / len(df_analysis)) * 100
    print(f"   {category:8s}: {count:4d} ({percentage:5.1f}%)")
print()

# 3. Correlation Insights
print("3. CORRELATION INSIGHTS")
print("-" * 40)
print(f"   Strongest positive correlation with charges:")
max_corr = charges_corr.idxmax()
print(f"     - {max_corr}: {charges_corr[max_corr]:+.4f}")
print()
print(f"   Top 3 factors influencing insurance charges:")
for i, (feature, corr) in enumerate(charges_corr.head(3).items(), 1):
    print(f"     {i}. {feature:20s}: {corr:+.4f}")
print()

# 4. Smoker Impact
print("4. SMOKER STATUS IMPACT")
print("-" * 40)
smoker_charges = df_analysis.groupby('smoker')['charges_original'].agg(['mean', 'median', 'std'])
print(f"   Non-smokers - Mean charges: ${smoker_charges.loc['no', 'mean']:,.2f}")
print(f"   Smokers     - Mean charges: ${smoker_charges.loc['yes', 'mean']:,.2f}")
smoker_diff = smoker_charges.loc['yes', 'mean'] - smoker_charges.loc['no', 'mean']
smoker_pct = (smoker_diff / smoker_charges.loc['no', 'mean']) * 100
print(f"   Difference: ${smoker_diff:,.2f} ({smoker_pct:+.1f}%)")
print()

# 5. Age and BMI Analysis
print("5. AGE AND BMI PATTERNS")
print("-" * 40)
risk_stats = df_analysis.groupby('risk_category')[['age', 'bmi']].mean().sort_index()
print("   Average values by risk category (normalized):")
for category in ['Low', 'Medium', 'High']:
    print(f"     {category:8s} - Age: {risk_stats.loc[category, 'age']:.3f}, BMI: {risk_stats.loc[category, 'bmi']:.3f}")
print()

# 6. Charges Statistics
print("6. INSURANCE CHARGES STATISTICS")
print("-" * 40)
charges_by_risk = df_analysis.groupby('risk_category')['charges_original'].agg(['mean', 'median', 'min', 'max'])
charges_by_risk = charges_by_risk.reindex(['Low', 'Medium', 'High'])
print("   Average charges by risk category:")
for category in ['Low', 'Medium', 'High']:
    mean_charge = charges_by_risk.loc[category, 'mean']
    print(f"     {category:8s}: ${mean_charge:,.2f}")
print()

# 7. Key Observations
print("7. KEY OBSERVATIONS")
print("-" * 40)
print("   • Smoking status is the strongest predictor of insurance charges")
print("   • Age shows moderate positive correlation with charges")
print("   • BMI has positive correlation, especially when combined with smoking")
print("   • High-risk individuals have significantly higher insurance costs")
print("   • The classification system effectively separates low, medium, and high-risk groups")
print()

# 8. Data Quality
print("8. DATA QUALITY")
print("-" * 40)
missing_count = df_analysis.isnull().sum().sum()
print(f"   Missing values: {missing_count}")
print(f"   Data completeness: {((1 - missing_count / df_analysis.size) * 100):.1f}%")
print(f"   Normalization: Applied to age, BMI, and children")
print(f"   Original charges preserved for interpretability")
print()

print("="*80)
print("EXPLORATORY DATA ANALYSIS COMPLETED")
print("="*80)


# End